In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

def association_r(df,min_support=0.3):
    df['date'] = pd.to_datetime(df['date'])

    basket = df.groupby(['date', 'type'])['qts'].sum().unstack().reset_index().fillna(0)

    basket = basket.set_index('date')
    basket = basket > 0

    frequent_itemsets = apriori(basket, min_support=min_support, use_colnames=True)

    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    
    return frequent_itemsets.to_string()+"\n"+rules.to_string()

In [2]:
df = pd.read_csv("data.csv")

df['transaction_id'] = df.index + 1

df['cout_achat'] = 0

print(association_r(df,min_support=0.3))

    support  itemsets
0  0.631944      (S1)
1  0.586806      (S2)
2  0.381944  (S2, S1)
  antecedents consequents  antecedent support  consequent support   support  confidence      lift  leverage  conviction  zhangs_metric
0        (S2)        (S1)            0.586806            0.631944  0.381944    0.650888  1.029976  0.011116    1.054261       0.070435
1        (S1)        (S2)            0.631944            0.586806  0.381944    0.604396  1.029976  0.011116    1.044464       0.079074


In [3]:
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objects as go

def predict_name(df, months=3, name='revenue'):
    # Calcul du revenu
    df['revenue'] = df['price_unit'] * df['qts']

    # Conversion de la colonne de date en format datetime
    df['date'] = pd.to_datetime(df['date'])

    # Agrégation des revenus par mois
    monthly_sales = df.groupby('date')[name].sum()

    print(monthly_sales)
    
    # Modèle SARIMA
    model = sm.tsa.SARIMAX(monthly_sales, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)

    # Prévisions pour les prochains mois
    forecast = model_fit.forecast(steps=months)
    forecast_dates = pd.date_range(start=monthly_sales.index[-1] + pd.DateOffset(months=1), periods=months, freq='M')
    forecast_series = pd.Series(forecast, index=forecast_dates)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=monthly_sales.index, y=monthly_sales, mode='lines', name='Ventes historiques', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=forecast_series.index, y=forecast_series, mode='lines', name='Prévisions SARIMA', line=dict(color='red')))

    fig.update_layout(
        title='Prévisions des ventes avec SARIMA',
        xaxis_title='Date',
        yaxis_title='Revenu total',
        legend_title='Légende',
        template='plotly',
    )

    fig.show()

predict_name(df,name='price_unit')

date
2024-01-01    -19
2024-01-02    260
2024-01-03    120
2024-01-04    170
2024-01-05     70
             ... 
2024-12-25    170
2024-12-26    170
2024-12-27    170
2024-12-28    170
2024-12-29     70
Name: price_unit, Length: 288, dtype: int64


c:\Users\ROUSTOUM\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\ROUSTOUM\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\ROUSTOUM\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\ROUSTOUM\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is availa

In [6]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
from PIL import Image
import io

def product_future_prediciton(df):
    df['date'] = pd.to_datetime(df['date'])

    df['revenue'] = df['price_unit'] * df['qts']
    df['cost_of_goods_sold'] = df['cout_achat'] * df['qts']
    df['gross_margin'] = df['revenue'] - df['cost_of_goods_sold']
    df['net_profit'] = df['revenue'] - df['cost_of_goods_sold']

    grouped_df = df.groupby(['type', pd.Grouper(key='date', freq='M')]).agg({
        'price_unit': 'sum',
        'cout_achat': 'sum',
        'qts': 'sum',
        'revenue': 'sum',
        'cost_of_goods_sold': 'sum',
        'gross_margin': 'sum',
        'net_profit': 'sum'
    }).reset_index()
    
    start_date = df['date'].min()
    end_date = df['date'].max()

    all_dates = pd.date_range(start=start_date, end=end_date, freq='M')
    all_products = df['type'].unique()
    all_combinations = pd.MultiIndex.from_product([all_products, all_dates], names=['type', 'date'])

    full_df = pd.DataFrame(index=all_combinations).reset_index()

    merged_df = full_df.merge(grouped_df, on=['type', 'date'], how='left')

    merged_df['price_unit'] = merged_df['price_unit'].fillna(0)
    merged_df['cout_achat'] = merged_df['cout_achat'].fillna(0)
    merged_df['qts'] = merged_df['qts'].fillna(0)
    merged_df['revenue'] = merged_df['revenue'].fillna(0)
    merged_df['cost_of_goods_sold'] = merged_df['cost_of_goods_sold'].fillna(0)
    merged_df['gross_margin'] = merged_df['gross_margin'].fillna(0)
    merged_df['net_profit'] = merged_df['net_profit'].fillna(0)
    
    monthly_margins = merged_df.groupby(['type', 'date'])['gross_margin'].sum().reset_index()

    sarima_predictions = {}
    for type, product_data in monthly_margins.groupby('type'):
        forecast = sarima_forecast(product_data)
        sarima_predictions[type] = forecast

    for type, forecast in sarima_predictions.items():
        print(f"Produit {type}:")
        print(forecast)
    list_of_images = plot_margins_separately(monthly_margins, sarima_predictions)

    return monthly_margins , sarima_predictions , product_data , list_of_images

    
        
def sarima_forecast(product_data):
    product_data.set_index('date', inplace=True)
    model = sm.tsa.SARIMAX(product_data['gross_margin'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    forecast = model_fit.forecast(steps=6)
    return forecast

def plot_margins_separately_plt(monthly_margins, sarima_predictions):
    a = []
    for i, type_ in enumerate(monthly_margins['type'].unique(), start=1):
        product_data = monthly_margins[monthly_margins['type'] == type_].set_index('date')
        
        plt.figure(figsize=(10, 6))
        
        plt.plot(product_data.index, product_data['gross_margin'], label='Marges brutes historiques', color='blue')
        
        if type_ in sarima_predictions:
            plt.plot(sarima_predictions[type_].index, sarima_predictions[type_], label='Prévisions SARIMA', color='red')
        

        plt.title(f'Prévisions des marges brutes pour le produit {type_}')
        plt.xlabel('Date')
        plt.ylabel('Marge brute')
        plt.legend()
        
        buf = io.BytesIO()
        plt.savefig(buf, format='jpg')
        buf.seek(0)

        image = Image.open(buf)
        
        image.show()
    
        plt.close()

        a.append(image)
    return a 

def plot_margins_separately(monthly_margins, sarima_predictions):
    for i, type_ in enumerate(monthly_margins['type'].unique(), start=1):

        product_data = monthly_margins[monthly_margins['type'] == type_].set_index('date')
        
        fig = go.Figure()

        fig.add_trace(go.Scatter(x=product_data.index, y=product_data['gross_margin'], mode='lines', 
                                 name='Marges brutes historiques', line=dict(color='blue')))

        if type_ in sarima_predictions:
            fig.add_trace(go.Scatter(x=sarima_predictions[type_].index, y=sarima_predictions[type_], mode='lines', 
                                     name='Prévisions SARIMA', line=dict(color='red')))

        fig.update_layout(
            title=f'Prévisions des marges brutes pour le produit {type_}',
            xaxis_title='Date',
            yaxis_title='Marge brute',
            legend_title='Légende',
            template='plotly',
        )

        fig.show()
    return None

monthly_margins , sarima_predictions , product_data , a =product_future_prediciton(df)
df

Produit I1:
2024-12-31   -136.00000
2025-01-31   -355.50011
2025-02-28   -242.50011
2025-03-31   -225.50011
2025-04-30   -102.50011
2025-05-31     11.49989
Freq: M, Name: predicted_mean, dtype: float64
Produit I2:
2024-12-31   -422.000000
2025-01-31   -813.000195
2025-02-28   -630.000195
2025-03-31   -540.000195
2025-04-30   -278.000195
2025-05-31   -300.000195
Freq: M, Name: predicted_mean, dtype: float64
Produit I3:
2024-12-31   -193.000000
2025-01-31   -306.000056
2025-02-28   -217.000056
2025-03-31   -194.000056
2025-04-30   -153.000056
2025-05-31   -112.000056
Freq: M, Name: predicted_mean, dtype: float64
Produit M:
2024-12-31    2.022139e-45
2025-01-31   -1.451626e-30
2025-02-28    2.278508e-15
2025-03-31   -5.000000e+02
2025-04-30    0.000000e+00
2025-05-31    2.136776e-14
Freq: M, Name: predicted_mean, dtype: float64
Produit S1:
2024-12-31     950.000000
2025-01-31    1725.000387
2025-02-28    1425.000387
2025-03-31    1525.000387
2025-04-30     725.000387
2025-05-31    1025.00

c:\Users\ROUSTOUM\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency M will be used.

c:\Users\ROUSTOUM\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency M will be used.

c:\Users\ROUSTOUM\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.

c:\Users\ROUSTOUM\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.

c:\Users\ROUSTOUM\AppData\Local\Programs\Python\Python3

,type,date,price_unit,qts,transaction_id,cout_achat,revenue,cost_of_goods_sold,gross_margin,net_profit
0,I1,2024-01-01,-12,10,1,0,-120,0,-120,-120
1,I2,2024-01-01,-5,17,2,0,-85,0,-85,-85
2,I3,2024-01-01,-2,18,3,0,-36,0,-36,-36
3,S2,2024-01-02,70,1,4,0,70,0,70,70
4,S2,2024-01-02,70,1,5,0,70,0,70,70
...,...,...,...,...,...,...,...,...,...,...
615,S1,2024-12-27,50,1,616,0,50,0,50,50
616,S2,2024-12-28,70,1,617,0,70,0,70,70
617,S1,2024-12-28,50,1,618,0,50,0,50,50
618,S1,2024-12-28,50,1,619,0,50,0,50,50


In [5]:
def plot_margins_separately_plt(monthly_margins, sarima_predictions):
    a = []
    for i, type_ in enumerate(monthly_margins['type'].unique(), start=1):
        # Filtrer les données pour chaque type de produit
        product_data = monthly_margins[monthly_margins['type'] == type_].set_index('date')
        
        # Créer une nouvelle figure pour chaque type de produit
        plt.figure(figsize=(10, 6))
        
        # Tracé des marges brutes historiques
        plt.plot(product_data.index, product_data['gross_margin'], label='Marges brutes historiques', color='blue')
        
        # Tracé des prévisions SARIMA, si elles existent pour ce type
        if type_ in sarima_predictions:
            plt.plot(sarima_predictions[type_].index, sarima_predictions[type_], label='Prévisions SARIMA', color='red')
        
        # Mise en forme du graphique
        plt.title(f'Prévisions des marges brutes pour le produit {type_}')
        plt.xlabel('Date')
        plt.ylabel('Marge brute')
        plt.legend()
        
        # Sauvegarde de l'image dans un buffer en mémoire pour la convertir en bytes
        buf = io.BytesIO()
        plt.savefig(buf, format='jpg')
        buf.seek(0)

        # Charger l'image avec PIL à partir du buffer
        image = Image.open(buf)
        
        # Affichage de l'image avec PIL
        image.show()
    
        # Fermer la figure Matplotlib après affichage
        plt.close()

        a.append(image)
    return a 